In [ ]:
pip install git+https://github.com/IDEA-Research/GroundingDINO.git

In [1]:
# Установи: pip install groundingdino-py
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt
from groundingdino.util.inference import load_model, load_image, predict

def simple_grounding_dino(image_path, text_prompt):
    """Простой GroundingDINO - только детекция по тексту"""
    
    print(f"🔍 Ищем: '{text_prompt}'")
    
    # 1. Загружаем модель (автоскачается при первом запуске)
    print("📥 Загружаем GroundingDINO...")
    model = load_model(
        "groundingdino/config/GroundingDINO_SwinT_OGC.py", 
        "weights/groundingdino_swint_ogc.pth"
    )
    
    # 2. Загружаем изображение
    image_source, image = load_image(image_path)
    
    # 3. Ищем объекты по тексту
    print("🎯 Ищем объекты...")
    boxes, scores, phrases = predict(
        model=model,
        image=image, 
        caption=text_prompt,
        box_threshold=0.35,    # Минимальная уверенность bbox
        text_threshold=0.25    # Минимальное соответствие тексту
    )
    
    # 4. Показываем что нашли
    print(f"✅ Найдено объектов: {len(boxes)}")
    for i, (phrase, score) in enumerate(zip(phrases, scores)):
        print(f"   {i+1}. {phrase} (уверенность: {score:.3f})")
    
    # 5. Рисуем результат
    image_display = image_source[0].permute(1, 2, 0).cpu().numpy()
    
    plt.figure(figsize=(12, 8))
    plt.imshow(image_display)
    
    # Рисуем bounding boxes
    for i, (box, phrase, score) in enumerate(zip(boxes, phrases, scores)):
        # Конвертируем координаты
        H, W = image_source.shape[1], image_source.shape[2]
        x1, y1, x2, y2 = box * torch.tensor([W, H, W, H])
        
        # Рисуем прямоугольник
        rect = plt.Rectangle((x1, y1), x2-x1, y2-y1, 
                           linewidth=3, edgecolor='red', facecolor='none')
        plt.gca().add_patch(rect)
        
        # Добавляем подпись
        label = f"{phrase} ({score:.2f})"
        plt.text(x1, y1-10, label, 
                fontsize=12, fontweight='bold', color='white',
                bbox=dict(boxstyle="round,pad=0.3", facecolor='red', alpha=0.8))
    
    plt.axis('off')
    plt.title(f"GroundingDINO: '{text_prompt}'\nНайдено: {len(boxes)} объектов", fontsize=14)
    plt.tight_layout()
    plt.show()

# 🎯 ПРОСТЫЕ ПРИМЕРЫ:

print("=== ПРИМЕР 1: Человек ===")
simple_grounding_dino(
    "/home/oleg/projects/ML/CLIP/images/0d354ad89e92986b19b10a8ac2797dfb.jpg",
    "person"
)

print("\n=== ПРИМЕР 2: Несколько объектов ===")
simple_grounding_dino(
    "/home/oleg/projects/ML/CLIP/images/0d354ad89e92986b19b10a8ac2797dfb.jpg", 
    "person . laptop . cup"
)

print("\n=== ПРИМЕР 3: Только чашка ===")
simple_grounding_dino(
    "/home/oleg/projects/ML/CLIP/images/0d354ad89e92986b19b10a8ac2797dfb.jpg",
    "cup"
)

ModuleNotFoundError: No module named 'groundingdino'